In [63]:
import numpy as np
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector
import dimod
import importlib

In [79]:
import qubo_params
importlib.reload(qubo_params)
Q = qubo_params.Q

print(f'len(Q): {len(Q)}')
print(max(Q.values()))

1662973014.2323494
len(Q): 402
2.0


In [80]:
relative_chain_strength = 1
chain_strength = max(Q.values()) * relative_chain_strength
print(f'chain strength: {chain_strength}')

n_runs = 100

chain strength: 2.0


In [81]:
use_qpu = True
if use_qpu:
    bqm = dimod.BQM.from_qubo(Q)
    sampler = EmbeddingComposite(DWaveSampler())
    response = sampler.sample(
      bqm,
      chain_strength=chain_strength,
      num_reads=n_runs,
      annealing_time=100,
      label='Protein Folding (QUBO)',
    )
else:
    response = dimod.ExactSolver().sample_qubo(Q)

In [83]:
print("Energy Reached:", response.first.energy)
print(f"Number of records: {len(response.record)}")
solutions = response.lowest()
print(f'Number of solutions: {len(solutions)}')
print(solutions.slice(0, 10))
print(response.first)

Energy Reached: -4.75
Number of records: 100
Number of solutions: 2
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 ... 35 energy num_oc. ...
0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0 ...  0  -4.75       1 ...
1  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  1  1 ...  0  -4.75       1 ...
['BINARY', 2 rows, 2 samples, 36 variables]
Sample(sample={0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 1, 28: 0, 29: 0, 30: 1, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0}, energy=-4.75, num_occurrences=1, chain_break_fraction=0.027777777777777776)


In [73]:
dwave.inspector.show(response)

'http://127.0.0.1:18000/?problemId=83cb2f29-cd27-4167-959e-987c2438818e'